# **Insper - Eletromagnetismo e Ondulatória** #


**Aula 9 - Transformador**

**Item 14** - Calculando correntes e tensões em um transformador AC



In [1]:
#Vamos importar os módulos que precisamos
from cmath import *
from numpy import linalg
import numpy as np

---
Na célula a seguir:

a) Declare os parâmetros de nosso transformador: $R_1=6\Omega$, $R_2=3\Omega$, $L_1=300mH$, $L_2=30mH$, frequência $f=60 Hz$ e tensão $V_1=50cos(\omega t)$.

b) Calcule:

- Frequência angular $\omega$
- Indutância Mútua $M$
- Reatâncias indutivas de $L_1$ e $L_2$, ou seja $XL1$ e $XL2$
- Reatância indutiva mútua $XM$

Obsrvação: o número complexo $j=\sqrt{-1}$ é nativo da linguagem Python. Para escrever $j$, digite "1j"

In [2]:
#indutâncias
L1=300e-3
L2=30e-3
#resistências
R1=6
R2=3
#frequência
f=60

#frequência angular
w=2*pi*f
#tensão da fonte (pico)
Ufonte=50
#indutância mútua
M=sqrt(L1*L2)
#reatâncias do transformador
XL1=1j*w*L1
XL2=1j*w*L2
XM=1j*w*M



c) Observe a função que criamos na célula abaixo. Tente explicar o que ela faz.

In [3]:
def CalcularTransformador(Uf, Rc):
    Z=np.array([[R1+XL1, -XM],[-XM, XL2+R2+Rc]])
    V=np.array([Uf,0])
    i=np.dot(linalg.inv(Z),V)
    return i[0], i[1]

d) Calcule as correntes $i_1$ e $i_2$ para três condições de carga ($R_C=1M\Omega$, $R_C=100\Omega$ e $R_C=1\Omega$). Assim que calcular as correntes, calcule também $V_2$ para cada resistor de carga (analise o circuito para saber como calculá-la). Por fim, calcule as relações $N_1/N_2$, $e_1/e_2$ e $V_1/V_2$ e, o mais importante: calcule as potências no primário ($S1$) e no secundário ($S2$).

IMPORTANTE: para dar print nas relações $N_1/N_2$, $V_1/V_2$ e $e_1/e_2$, você precisará usar o **módulo dos fasores**. Para calcular o módulo de um número complexo, use "np.abs(número complexo)".

In [4]:
listaRc=[1e6, 100, 1]

for Rc in listaRc:
    i1, i2 = CalcularTransformador(Ufonte, Rc)
    V2=i2*Rc
    V1=Ufonte
    e1=-XL1*i1+XM*i2
    e2=-XM*i1+XL2*i2  
    S1=V1/sqrt(2)*i1.conjugate()/sqrt(2)
    S2=V2/sqrt(2)*i2.conjugate()/sqrt(2)
    print('Com resistor na saída de %.2f Ω:' %Rc)
    print('i1 (pico) = %.2f A' %np.abs(i1))
    print('i2 (pico) = %.2f A' %np.abs(i2))
    print('V1 (pico) = %.2f V' %Ufonte)    
    print('V2 (pico) = %.2f V' %(np.abs(V2)))
    print('Relação N1/N2 = %.2f' %(np.abs(sqrt(L1/L2))))
    print('Relação e1/e2 = %.2f' %(np.abs(e1/e2)))
    print('Relação V1/V2 = %.2f' %(np.abs(V1/(V2))))
    print('S1 = %.2f +j%.2f' %(S1.real,S1.imag))
    print('S2 = %.2f +j%.2f' %(S2.real,S2.imag))
    print('Eficiência = %.2f' %(S2.real/S1.real))
    print('---')


Com resistor na saída de 1000000.00 Ω:
i1 (pico) = 0.44 A
i2 (pico) = 0.00 A
V1 (pico) = 50.00 V
V2 (pico) = 15.79 V
Relação N1/N2 = 3.16
Relação e1/e2 = 3.16
Relação V1/V2 = 3.17
S1 = 0.58 +j11.02
S2 = 0.00 +j0.00
Eficiência = 0.00
---
Com resistor na saída de 100.00 Ω:
i1 (pico) = 0.44 A
i2 (pico) = 0.15 A
V1 (pico) = 50.00 V
V2 (pico) = 15.24 V
Relação N1/N2 = 3.16
Relação e1/e2 = 3.16
Relação V1/V2 = 3.28
S1 = 1.78 +j10.89
S2 = 1.16 +j-0.00
Eficiência = 0.65
---
Com resistor na saída de 1.00 Ω:
i1 (pico) = 1.15 A
i2 (pico) = 3.43 A
V1 (pico) = 50.00 V
V2 (pico) = 3.43 V
Relação N1/N2 = 3.16
Relação e1/e2 = 3.16
Relação V1/V2 = 14.56
S1 = 27.56 +j8.34
S2 = 5.89 +j0.00
Eficiência = 0.21
---


e) Recalcule o item **d)**, mas agora considerando que o transformador é **ideal**. Para isso, basta zerar as resistências R1 e R2 do primário e do secundário. Após zer-a-las, copie o código do item d) na célula abaixo. O que você pode concluir sobre a potência real nas três condições de carga?

In [5]:
R1=0
R2=0

listaRc=[1e6, 100, 1]

for Rc in listaRc:
    i1, i2 = CalcularTransformador(Ufonte, Rc)
    V2=i2*Rc
    V1=Ufonte
    e1=-XL1*i1+XM*i2
    e2=-XM*i1+XL2*i2  
    S1=V1/sqrt(2)*i1.conjugate()/sqrt(2)
    S2=V2/sqrt(2)*i2.conjugate()/sqrt(2)
    print('Com resistor na saída de %.2f Ω:' %Rc)
    print('i1 (pico) = %.2f A' %np.abs(i1))
    print('i2 (pico) = %.2f A' %np.abs(i2))
    print('V1 (pico) = %.2f V' %Ufonte)    
    print('V2 (pico) = %.2f V' %(np.abs(V2)))
    print('Relação N1/N2 = %.2f' %(np.abs(sqrt(L1/L2))))
    print('Relação e1/e2 = %.2f' %(np.abs(e1/e2)))
    print('Relação V1/V2 = %.2f' %(np.abs(V1/(V2))))
    print('S1 = %.2f +j%.2f' %(S1.real,S1.imag))
    print('S2 = %.2f +j%.2f' %(S2.real,S2.imag))
    print('Eficiência = %.2f' %(S2.real/S1.real))
    print('---')


Com resistor na saída de 1000000.00 Ω:
i1 (pico) = 0.44 A
i2 (pico) = 0.00 A
V1 (pico) = 50.00 V
V2 (pico) = 15.81 V
Relação N1/N2 = 3.16
Relação e1/e2 = 3.16
Relação V1/V2 = 3.16
S1 = 0.00 +j11.05
S2 = 0.00 +j0.00
Eficiência = 1.00
---
Com resistor na saída de 100.00 Ω:
i1 (pico) = 0.44 A
i2 (pico) = 0.16 A
V1 (pico) = 50.00 V
V2 (pico) = 15.81 V
Relação N1/N2 = 3.16
Relação e1/e2 = 3.16
Relação V1/V2 = 3.16
S1 = 1.25 +j11.05
S2 = 1.25 +j0.00
Eficiência = 1.00
---
Com resistor na saída de 1.00 Ω:
i1 (pico) = 5.02 A
i2 (pico) = 15.81 A
V1 (pico) = 50.00 V
V2 (pico) = 15.81 V
Relação N1/N2 = 3.16
Relação e1/e2 = 3.16
Relação V1/V2 = 3.16
S1 = 125.00 +j11.05
S2 = 125.00 +j0.00
Eficiência = 1.00
---
